In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


global df

cols=['product_block', 'TPNB', 'description', 'cost_component',
   'current_cost', 'quoted_cost', 'best_bid', 'best_supplier', 'volume',
   'Impact']
cols_replacement=['Product Block','TPNB','Description','Cost Component','Current Cost','Quoted Cost','Best Bid','Best Supplier','Volume','Impact']

suppliers_list=list(df['supplier_name'].unique())
suppliers_list

product_block_list=list(df['product_block'].unique())
product_block_list

PB_cols = suppliers_list.copy()
PB_cols.insert(0,'Product Block')
PB_cols.insert(len(PB_cols),'Minimum')

supplier_feedback_cols1=['supplier_name','product_block','TPNB','Total_Product_Bid%']
supplier_feedback_cols1_replacement=['Supplier Name','Product Block','TPNB','Total Product Bid %']


supplier_feedback_cols2=['product_block', 'TPNB', 'description', 'volume','Supplier_Quantile_Rank', 'Position']
supplier_feedback_cols2_replacment=['Product Block', 'TPNB', 'Description', 'Volume','Supplier Quantile Rank', 'Position']


supplier_feedback_cols3=['product_block','Bid_%','competitive','reason']
supplier_feedback_cols3_replacement=['Product Block','Bid %','Competitive','Reason']

block_grid_cols=['product_block',
 'TPNB',
 'volume',
 'negotiation_index',
 'current_cost',
 'No_Bids',
 'best_supplier',
 'vol_avg',
 'NI_Avg',
 'Action_Priority',
 'lowest_bidder_cost',
 'Risk',
 'max_bid_supplier']
block_grid_cols.extend(suppliers_list)


block_grid_cols_replacement=['Product Block',
 'TPNB',
 'Volume',
 'Negotiation Index',
 'Current Cost',
 'No Bids',
 'Best Supplier',
 'Vol Avg',
 'NI Avg',
 'Action Priority',
 'Lowest Bidder Cost',
 'Risk',
 'Max Bid Supplier']
block_grid_cols_replacement.extend(suppliers_list)

resets=['Vinegar','Salmon']
app=dash.Dash()






external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
#     'fontWeight': 'bold',
    'color':'#000'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#00539f',
    'color': 'white',
    'padding': '6px'
}
headings={'color':'#00539f','background-color':'#fff','text-align':'center'}


app.layout=html.Div([
    html.Div([],style={'background-color':'#00539f','height':'5px','width':'100%'}),
    
    html.Div([
        html.Img(src="https://www.logosurfer.com/wp-content/uploads/2018/03/tesco-logo_0.png",style={'float':'left','height':'20px','padding':'2px'}),
        dcc.RadioItems(id="resets-dropdown",
                            options=[
                                {'label': i, 'value': i} for i in resets
                            ],
                            value=resets[0],
                            style={'background-color':'#fff','color':'#000'}
                     
                        ),
        html.Label("Bid Summary Tool - Supplier Analysis",style=headings),
        html.Div(id='reset_name'),
        
    ],style={'height':'140px','background-color':'#fff'}
    ),
    
    html.Div([
    dcc.Tabs(id='tabs', value='tab-1', children=[
        dcc.Tab(label='Summary', value='tab-4',style=tab_style, selected_style=tab_selected_style,children=[]),
        dcc.Tab(label='Supplier', value='tab-1',style=tab_style, selected_style=tab_selected_style,children=[
            dcc.Dropdown(id="supplier-dropdown",
                    options=[
                        {'label': i, 'value': i} for i in suppliers_list
                    ],
                    value=suppliers_list[0]
                     
            ),
            
            html.Div([
                html.Div([
                dcc.Graph(id='plot')
                ],style={'display':'inline-block','width':'50%'}),
            
                html.Div([
                    dcc.Graph(id='plot_2')
                    ],style={'display':'inline-block','width':'50%'}),
                ],style={'width': '100%', 'height': '450px'}
            ),
            html.Label("Top Oppurtunities",style={'background-color':'#ccc','color':'#000','text-align':'center'}),
            dash_table.DataTable(
                    id='computed-table',
                    columns=[
                            {"name": i, "id": i} for i in cols_replacement
                        ],
                    style_as_list_view=False,
                    style_cell={'padding': '5px'},
                    style_header={
                        'backgroundColor': '#00539f',
                        'fontWeight': 'bold',
                        'color':'#fff'
                    },
                    style_data={
                        'color':'#00539f'
                    }
        
            ),
            html.Div([
                    html.Div([
                        dcc.Dropdown(
                            id="product_block_dropdown",
                            multi=True,
                            placeholder = "Select a Product Block",
                            # options = [
                            #     {"label": i , "value": i} for i in block_list
                            # ],
                            # value = block_list[0],
                            style={'background-color': '#C0C0C0', 'color': '#000'}
                        ),
                    ], style={'display': 'inline-block', 'width': '30%'}),
                    html.Div([
                        dcc.Dropdown(
                            id="TPNB_dropdown",
                            multi=True,
                            placeholder = "Select a TPNB",
                            style={'background-color': '#C0C0C0', 'color': '#000'}
                        ),
                    ], style={'display': 'inline-block', 'width': '70%'}),
                ]),
                html.Div([
                        html.Div([
                            dcc.Graph(
                                id="Quoted_Price_Comparison",
                                style={'height': '400px'}
                            )
                        ], style={'display': 'inline-block', 'width': '50%'}),

                        html.Div([
                            dcc.Graph(
                                id="Total_Price_Comparison",
                                style={'height': '400px'}
                            )
                        ], style={'display': 'inline-block', 'width': '50%'}),

                    ], style={'width': '100%', 'height': '400px'}
                ),
            
                ]
        ),
        dcc.Tab(label='Block', value='tab-2',style=tab_style, selected_style=tab_selected_style,children=[
            dcc.Dropdown(id="product_block-dropdown",
                         multi=True,
                    options=[
                        {'label': i, 'value': i} for i in product_block_list
                    ],
                    value=product_block_list,
                    style={'background-color':'#000','color':'#000'}
                     
            ),
            html.Div([
                dcc.Graph(id='Supplier Coverage plot')
                ],style={'display':'inline-block','width':'100%'}
            ),
            html.Label("Block Grid",style={'background-color':'#ccc','color':'#000','text-align':'center'}),
            dash_table.DataTable(
                    id='block_table',
                    columns=[
                            {"name": i, "id": i} for i in list(block_grid_cols_replacement)
                        ],
                    style_as_list_view=False,
                    style_cell={'padding': '5px'},
                    style_header={
                        'backgroundColor': '#00539f',
                        'fontWeight': 'bold',
                        'color':'#fff'
                    },
                    style_data={
                        'color':'#00539f'
                    },
                style_table={'overflowX': 'scroll'},
                style_data_conditional=[
                    {
                               'if': {
                                        'column_id': 'Risk',
                                        'filter_query': '{Risk} eq "Yes"'
                                    },
                                      'backgroundColor': '#EE1C2E',
                                      'color': 'white'            
                    },
                    {
                        'if': {
                            'column_id': 'Risk',
                            'filter_query': '{Risk} eq "No"'
                                },
                                'backgroundColor': '#00B050',
                                'color': 'white',
                    },
                    {
                               'if': {
                                        'column_id': 'Action_Priority',
                                        'filter_query': '{Action_Priority} eq "HIGH"'
                                    },
                                      'backgroundColor': '#00B050',
                                      'color': 'white'            
                    },
                    {
                        'if': {
                            'column_id': 'Action_Priority',
                            'filter_query': '{Action_Priority} eq "LOW"'
                                },
                                'backgroundColor': '#EE1C2E',
                                'color': 'white',
                    },
                    {
                        'if': {
                            'column_id': 'Action_Priority',
                            'filter_query': '{Action_Priority} eq "MEDIUM"'
                                },
                                'backgroundColor': '#FFA826',
                                'color': 'white',
                    }
                    
                ]
        
            ),
            html.Div([
        dcc.Dropdown(
            id="pb_dropdown",
            multi=True,
            options=[
                {"label": i, "value": i} for i in product_block_list
            ],
            value=product_block_list,
            style={'background-color': '#C0C0C0', 'color': '#000'}
        ),
    ], style={'display': 'inline-block', 'width': '70%'}),

    html.Div([
        html.H4(children="Delivered Cost (Weighted average)"),
        dash_table.DataTable(
            id="Del_Cost_PB",
            columns=[{"name": i, "id": i} for i in PB_cols],
            style_as_list_view=False,
            style_cell={'padding': '5px'},
            style_header={
                'backgroundColor': '#00539f',
                'fontWeight': 'bold',
                'color':'#fff'
            },
            style_data={
                'color':'#00539f'
            },
        ),
    ]),

    html.Div([
        html.H4(children="Total Cost"),
        dash_table.DataTable(
            id="Total_Cost_PB",
            columns=[{"name": i, "id": i} for i in PB_cols],
            style_as_list_view=False,
            style_cell={'padding': '5px'},
            style_header={
                'backgroundColor': '#00539f',
                'fontWeight': 'bold',
                'color':'#fff'
            },
            style_data={
                'color':'#00539f'
            },
        ),
    ]),

    html.Div([
        html.H4(children="Savings"),
        dash_table.DataTable(
            id="Savings_PB",
            columns=[{"name": i, "id": i} for i in PB_cols],
            style_as_list_view=False,
            style_cell={'padding': '5px'},
            style_header={
                'backgroundColor': '#00539f',
                'fontWeight': 'bold',
                'color':'#fff'
            },
            style_data={
                'color':'#00539f'
            },
        ),
    ])
            
            
        ]),
dcc.Tab(label='Supplier Feedback', value='tab-3',style=tab_style, selected_style=tab_selected_style,children=[
            html.Div([
                html.Div(style={'float':'left','width':'20%'},children=[
                    dcc.RadioItems(id="supplier-checklist",
                            options=[
                                {'label': i, 'value': i} for i in suppliers_list
                            ],
                            value=suppliers_list[0],
                            style={'background-color':'#fff','color':'#000'}
                     
                        )
                    
                ])
                
            ]),
            html.Div([
                    dash_table.DataTable(
                        id="Supplier_Feedback",
                        columns=[{"name": i, "id": i} for i in supplier_feedback_cols1_replacement],
                        style_as_list_view=False,
                        style_cell={'padding': '5px'},
                        style_header={
                            'backgroundColor': '#00539f',
                            'fontWeight': 'bold',
                            'color':'#fff'
                        },
                        style_data={
                            'color':'#00539f'
                        }
                    )
                    
                ],style={'float':'right','width':'80%'}
            ),
        html.Div(id="Overall_Position",
                 
                 
        ),
        html.Div(
            html.Label(children="Position Reason",style={"float":"left"}),
                        id="Reason",
        ),
        html.Div([
            dash_table.DataTable(
            id="Product_Block_Position",
                columns=[{"name": i, "id": i} for i in supplier_feedback_cols3_replacement],
                style_as_list_view=False,
                    style_cell={'padding': '5px'},
                style_header={
                        'backgroundColor': '#00539f',
                        'fontWeight': 'bold',
                        'color':'#fff'
                    },
                    style_data={
                        'color':'#00539f'
                    },
                style_data_conditional=[
                    {
                               'if': {
                                        'column_id': 'Competitive',
                                        'filter_query': '{Competitive} eq "Non Competitive"'
                                    },
                                      'backgroundColor': '#EE1C2E',
                                      'color': 'white'            
                    },
                    {
                        'if': {
                            'column_id': 'Competitive',
                            'filter_query': '{Competitive} eq "Competitive"'
                                },
                                'backgroundColor': '#00B050',
                                'color': 'white',
                    },
                {
                        'if': {
                            'column_id': 'Competitive',
                            'filter_query': '{Competitive} eq "Competitive_2"'
                                },
                                'backgroundColor': '#00B050',
                                'color': 'white',
                    }]
                
                
                
            )
        ]),
           html.Div([
                html.H4(children="Product Level Position"),
                dash_table.DataTable(
                    id="supplier_feedback_grid",
                    columns=[{"name": i, "id": i} for i in supplier_feedback_cols2_replacment],
                    style_as_list_view=False,
                    style_cell={'padding': '5px'},
                    style_header={
                        'backgroundColor': '#00539f',
                        'fontWeight': 'bold',
                        'color':'#fff'
                    },
                    style_data={
                        'color':'#00539f'
                    },
                    style_data_conditional=[
                    {
                               'if': {
                                        'column_id': 'Position',
                                        'filter_query': '{Position} eq "NON COMPETITIVE"'
                                    },
                                      'backgroundColor': '#EE1C2E',
                                      'color': 'white'            
                    },
                    {
                        'if': {
                            'column_id': 'Position',
                            'filter_query': '{Position} eq "COMPETITIVE"'
                                },
                                'backgroundColor': '#00B050',
                                'color': 'white',
                    },
                    {
                               'if': {
                                        'column_id': 'Position',
                                        'filter_query': '{Position} eq "NOT BID"'
                                    },
                                      'backgroundColor': '#FFA826',
                                      'color': 'white'            
                    }
                    
                ]
                ),
    ]),
    
    ]),
    
    ]),
    html.Div(id='tabs-content')
    ]),
    
],style={'font-family': 'Tesco'})

@app.callback(
    Output("reset_name","children"),
    [Input("resets-dropdown","value")])
def reset_data(ireset):
    global df
    if(ireset=='Salmon'):
        df=pd.read_excel(r'C:\Users\IN22912959\OneDrive - Tesco\Documents\Resets\Final_Pivot_Data_Salmon_Phase2_v1.xlsx')
    if(ireset=='Vinegar'):
        df=pd.read_excel(r'C:\Users\IN22912959\OneDrive - Tesco\Documents\Resets\Final_Pivot_Data_SBR_Vinegar_Phase1_v1.xlsx')
    
    

    return 'You have selected "{}"'.format(ireset)
    


@app.callback(
    [Output("Supplier_Feedback","data"),
     Output("Overall_Position","children"),
    Output("Reason","children"),
    Output("Product_Block_Position","data")],
    [Input("supplier-checklist","value")])
def supplier_feedback_details(isupplier):
    
    cols=['supplier_name','product_block','TPNB','volume','negotiation_index','current_cost','No_Bids','best_supplier','Supplier_Quantile_Rank']
    df3=df[cols][(df['supplier_name']==isupplier)&(df['cost_component']=='Delivered Cost')&(df['Is_TPNB']==1)].drop_duplicates()
    product_block_count=df3.product_block.nunique()

    df4=df3.groupby(['supplier_name'])['product_block','TPNB'].nunique().reset_index()

    df['TPNB'].nunique()

    df4['Total_Product_Bid%']=round((df4['TPNB']/df['TPNB'].nunique())*100,0)

    upper_quartile=df3[(df3['Supplier_Quantile_Rank'].isin([1,2]))&(df3['supplier_name']==isupplier)]
    upper_quartile=upper_quartile.groupby(['product_block'])['Supplier_Quantile_Rank'].count().reset_index()
    upper_quartile

    lower_Quartile=df3[(df3['Supplier_Quantile_Rank'].isin([3,4]))&(df3['supplier_name']==isupplier)]
    lower_Quartile=lower_Quartile.groupby(['product_block'])['Supplier_Quantile_Rank'].count().reset_index()
    lower_Quartile


    upper_quartile_tpnb=sum(upper_quartile['Supplier_Quantile_Rank'])

    lower_quartile_tpnb=sum(lower_Quartile['Supplier_Quantile_Rank'])

    top_2_quartile_per=(upper_quartile_tpnb/(upper_quartile_tpnb+lower_quartile_tpnb))*100

    df4['Overall_position']=np.where(((df4['Total_Product_Bid%']>60)&(top_2_quartile_per>25)),'Competitive',
                                     np.where(((df4['Total_Product_Bid%']>40)&(top_2_quartile_per>40)),'Competitive_2','Non Competitive'))
    
    df4['Reason']=np.where(((df4['Total_Product_Bid%']>60) & (top_2_quartile_per>25)),'You bid on more than 60% of the block and more than 25% of your bids were competitive',
                                 np.where(((df4['Total_Product_Bid%']>40) & (top_2_quartile_per>40)),'You have not bid on a high % of products but more than 40% of your bids are competitive',
                                        np.where((df4['Total_Product_Bid%']<40),'You have bid on a very low number of products','A high percentage of your bids are not in the competitive zone')))
    df4
    cols=['supplier_name', 'product_block', 'TPNB', 'Total_Product_Bid%']
    df5=df4[cols]
    reason=df4['Reason'].reset_index()
    
    
    quartile=upper_quartile.merge(lower_Quartile,on=['product_block'],how='outer')

    quartile['upper_bid%']=((quartile['Supplier_Quantile_Rank_x'])/(quartile['Supplier_Quantile_Rank_x']+quartile['Supplier_Quantile_Rank_y']))*100

    tpnb_bid=df3.groupby(['product_block'])['TPNB'].nunique().reset_index()
    tpnb_bid

    total_tpnb=df.groupby(['product_block'])['TPNB'].nunique().reset_index()
    total_tpnb

    bid_per=tpnb_bid.merge(total_tpnb,how='inner',on=['product_block'])

    bid_per['Bid_%']=(bid_per['TPNB_x']/bid_per['TPNB_y'])*100
    bid_per

    product_block_position=quartile.merge(bid_per[['product_block','Bid_%']],on=['product_block'],how='outer')
    product_block_position

    product_block_position['competitive']=np.where(((product_block_position['Bid_%']>=80) & (product_block_position['upper_bid%']>=25)),'Competitive',
                                             np.where(((product_block_position['Bid_%']>=60) & (product_block_position['upper_bid%']>=40)),'Competitive_2','Non Competitive'))



    product_block_position['reason']=np.where(product_block_position['competitive']=='Competitive','You bid on more than 80% of the block and more than 25% of your bids were competitive',
                                     np.where(product_block_position['competitive']=='Competitive_2','You bid on less than 60% of the block and less than 40% of them were competitive','You bid on more than 80% of the block but less than 25% of your bids were competitive'))
    
    cols=['product_block','Bid_%','competitive','reason']
    product_block_position=product_block_position[cols]
    pbs=df['product_block'].drop_duplicates().to_frame()
    product_block_position=pbs.merge(product_block_position,on=['product_block'],how='left')
    product_block_position['Bid_%']=product_block_position['Bid_%'].fillna(0)
    product_block_position['competitive']=product_block_position['competitive'].fillna('Non Competitive')
    product_block_position['reason']=product_block_position['reason'].fillna('You bid on less than 60% of the block and less than 40% of them were competitive')
    product_block_position
    product_block_position.columns=['Product Block','Bid %','Competitive','Reason']
    new_cols=['Supplier Name','Product Block','TPNB','Total Product Bid %']
    df5.columns=new_cols
    return df5.to_dict('records'),[html.Div([html.P(children="Overall Position : "+df4['Overall_position'][0],style={'color': '#00539f', 'fontSize':'20px','background-color':'#ccc','padding-top':'82px'})])],[html.P(children="Position Reason:"+ df4['Reason'][0] ,style={'color': '#00539f', 'fontSize': '20px','background-color':'#ccc','padding-top':'20px'})],product_block_position.round(2).to_dict('records')




@app.callback(
    Output('plot', 'figure'),
    [Input('supplier-dropdown', 'value')])
def callback_color(prod):
    df_sup=df[(df['supplier_name']==prod)&(df['cost_component']=='Delivered Cost')]
    pivot = df_sup[['product_block','Is_TPNB','TPNB']].pivot_table(index=['product_block'],columns=['Is_TPNB'],aggfunc='count')
    pivot=pivot.fillna(0)
    pivot.columns = pivot.columns.droplevel(0)
    pivot_2 = pivot.reset_index().rename_axis(None, axis=1)
    fig = px.bar(pivot_2, x='product_block', y=1,title="TPN Coverage %",labels={
                     "1": "No of TPN's",
                     "product_block": "Product Block"
                 })

    
    return fig



@app.callback(
    Output('plot_2', 'figure'),
    [Input('supplier-dropdown', 'value')])
def savings(prod):
    df_sup=df[(df['supplier_name']==prod)&(df['cost_component']=='Delivered Cost')&(df['Is_NPD']==0)&(df['Is_TPNB']==1)]
    pivot = df_sup[['product_block','Savings']].pivot_table(index=['product_block'],aggfunc='sum')
    pivot=pivot.fillna(0)
    pivot_2 = pivot.reset_index().rename_axis(None, axis=1)
    pivot_2.head(2)
    pivot_2["Color"] = np.where(pivot_2["Savings"]>=0, 'green', 'red')

    fig = go.Figure()
    fig.add_trace(
        go.Bar(x=pivot_2['Savings'],
            y=pivot_2['product_block'],
            marker_color=pivot_2['Color'],
            orientation='h'
            ))
    fig.update_layout(title_text='Increment over current COGS',barmode='stack')

    return fig
    


@app.callback(
    Output('computed-table', 'data'),
    [Input('supplier-dropdown', 'value')])
def top_oppurtunities(prod):
    cols=['supplier_name','product_block','TPNB','description','Is_NPD','Is_TPNB','cost_component','current_cost','quoted_cost','best_bid','best_supplier','volume','Impact']
    df_sup=df[cols].drop_duplicates()
    df_sup_2=df_sup[(df['supplier_name']==prod)&(df_sup['Is_NPD']==0)&(df_sup['Is_TPNB']==1)]
    pivot = df_sup_2.groupby(['TPNB','cost_component'],as_index=False).agg({
        'current_cost':'sum',
        'quoted_cost':'sum',
        'best_bid':'sum',
        'Impact':'sum'

    }).reset_index(drop = True)
    df_2=pivot.sort_values(by=['Impact'],ascending=False).head(14)
    df3=df_2[['TPNB']].merge(df_sup[df_sup['supplier_name']==prod],how='inner',left_on=['TPNB'],right_on=['TPNB'])
    cols=['product_block','TPNB','description','cost_component','current_cost','quoted_cost','best_bid','best_supplier','volume','Impact']
    df4=df3[cols].drop_duplicates()
    df5=df4.sort_values(by=['Impact'],ascending=False).head(14)
    df5['Impact']=df5['Impact'].round(1)
    new_cols=['Product Block','TPNB','Description','Cost Component','Current Cost','Quoted Cost','Best Bid','Best Supplier','Volume','Impact']
    df5.columns=new_cols
    return df5.round(2).to_dict('records')
    




@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H3('Supplier Leve Bid Analysis')
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Block Level Bid Analysis')
        ])
    elif tab == 'tab-3':
        return html.Div([
            html.H3('Supplier Feedback')
        ])
    elif tab == 'tab-4':
        return html.Div([
            html.H3('Summary')
        ])



@app.callback(
    Output('Supplier Coverage plot', 'figure'),
    [Input('product_block-dropdown', 'value')])
def supplier_coverage_plot(prod):
    df_sup=df[(df['product_block'].isin(prod))&(df['cost_component']=='Delivered Cost')]
    pivot_Block_Covg = df_sup[['supplier_name','product_block','Is_TPNB','TPNB']].pivot_table(index=['supplier_name','product_block'],columns=['Is_TPNB'],aggfunc='count')
    pivot_Block_Covg = pivot_Block_Covg.fillna(0)
    pivot_Block_Covg.columns = pivot_Block_Covg.columns.droplevel(0)
    pivot_Block_Covg = pivot_Block_Covg.reset_index().rename_axis(None, axis=1)
    if(0 in pivot_Block_Covg.columns):
        pivot_Block_Covg['0_perc'] = pivot_Block_Covg[0] * 100 / (pivot_Block_Covg[0] + pivot_Block_Covg[1])
        pivot_Block_Covg['1_perc'] = pivot_Block_Covg[1] * 100 / (pivot_Block_Covg[0] + pivot_Block_Covg[1])
    else:
        pivot_Block_Covg['1_perc'] = pivot_Block_Covg[1] * 100 / (pivot_Block_Covg[1])


    fig = px.bar(pivot_Block_Covg, x='supplier_name', y='1_perc',title="Supplier Coverage %",
                 color="product_block",
                 barmode="group",
                 labels={
                         "1_perc": "% Bid",
                         "supplier_name": "Supplier Name",
                         "product_block":"Product Block"
                     })

    return fig
#     df_sup=df[(df['product_block']==prod)&(df['cost_component']=='Delivered Cost')]
#     pivot = df_sup[['supplier_name','product_block','Is_TPNB','TPNB']].pivot_table(index=['supplier_name','product_block'],columns=['Is_TPNB'],aggfunc='count')
#     pivot.columns = pivot.columns.droplevel(0)
#     pivot_2 = pivot.reset_index().rename_axis(None, axis=1)

#     cols = [0, 1]
#     pivot_2[cols] = pivot_2[cols].div(pivot_2[cols].sum(axis=1), axis=0).multiply(100)
#     pivot_2.fillna(0)
#     fig = px.bar(pivot_2, x='supplier_name', y=1,title="Supplier Coverage %",
#                  color="product_block",
#                  barmode="group",
#                  labels={
#                          "1": "%",
#                          "supplier_name": "Supplier Name",
#                          "product_block":"Product Block"
#                      })
    

    
#     return fig

@app.callback(
    Output('block_table', 'data'),
    [Input('product_block-dropdown', 'value')])
def block_grid(prod):
    cols=['product_block','TPNB','volume','negotiation_index','current_cost','No_Bids','best_supplier']
    df2=df[cols][(df['product_block'].isin(prod))&(df['cost_component']=='Delivered Cost')&(df['Is_TPNB']==1)].drop_duplicates()
    df2=df2.sort_values(by=['negotiation_index'],ascending=False)
    vol_avg=[]
    for i in range(0,len(df2)):
        avg=df2.iloc[i:]['volume'].mean()
        vol_avg.append(avg)

    df2['vol_avg']=vol_avg
    df2['NI_Avg']=df2['negotiation_index'].mean()
    df2['Action_Priority'] = np.where(((df2['volume']>df2['vol_avg']) & (df2['negotiation_index']>df2['NI_Avg'])),'HIGH',np.where(((df2['volume']<df2['vol_avg']) & (df2['negotiation_index']<df2['NI_Avg'])),'LOW','MEDIUM'))

    lowest_bid=df[(df['quoted_cost']!=0)&(df['cost_component']=='Delivered Cost')].groupby(['TPNB']).agg({'quoted_cost':'min'}).reset_index()
    lowest_bid=lowest_bid.rename(columns={'quoted_cost':'lowest_bidder_cost'})


    df2=df2.merge(lowest_bid,how='inner',on=['TPNB'])
    df2['Risk'] = np.where(((df2['lowest_bidder_cost']>df2['current_cost']) | (df2['No_Bids']<2)),'Yes','No')

    max_bid=df[(df['quoted_cost']!=0)&(df['cost_component']=='Delivered Cost')].groupby(['TPNB']).agg({'quoted_cost':'max'}).reset_index()
    max_bid=max_bid.merge(df[['TPNB','quoted_cost','supplier_name']],how='inner',on=['quoted_cost','TPNB'])
    max_bid=max_bid.rename(columns={'quoted_cost':'max_bidder_cost',
                                   'supplier_name':'max_bid_supplier'})
    df2=df2.merge(max_bid[['TPNB','max_bid_supplier']],how='inner',on=['TPNB'])

    block_grid=df[(df['cost_component']=='Delivered Cost')].groupby(['TPNB','supplier_name']).agg({'Is_TPNB':'mean'}).reset_index()
    block_grid=block_grid.sort_values(by=['Is_TPNB'],ascending=False)
    block_grid['Is_TPNB']=block_grid['Is_TPNB']*100
    block_grid = block_grid.pivot_table(index=['TPNB'],columns=['supplier_name'])
    block_grid.columns = block_grid.columns.droplevel(0)
    block_grid = block_grid.reset_index().rename_axis(None, axis=1)


    df2=df2.merge(block_grid,how='inner',on=['TPNB']).round(2) 
    new_cols=['Product Block',
              'TPNB',
              'Volume',
              'Negotiation Index',
              'Current Cost',
              'No Bids',
              'Best Supplier',
              'Vol Avg',
              'NI Avg',
              'Action Priority',
              'Lowest Bidder Cost',
              'Risk',
              'Max Bid Supplier']
    new_cols.extend(suppliers_list)
    df2.columns=new_cols
    
    return df2.round(2).to_dict('records')
    
    
@app.callback(
    Output("product_block_dropdown", "options"),
    [Input("supplier-dropdown", "value")]
)
def Product_block_List(iSupplier):
    block_supp_list = list(
        df[(df['supplier_name'] == iSupplier) & (df['Is_TPNB'] == 1)]['product_block'].unique())
    return [{'label': i, 'value': i} for i in block_supp_list]


@app.callback(
    Output("TPNB_dropdown", "options"),
    [Input("supplier-dropdown", "value"),
     Input("product_block_dropdown", "value")]
)
def TPNB_List(iSupplier, iBlock):
    TPNB_block_supp_list = list(df[(df['supplier_name'] == iSupplier) &
                                         (df['product_block'].isin(iBlock)) & (df['Is_TPNB'] == 1)][
                                    'TPNB'].unique())
    return [{'label': i, 'value': i} for i in TPNB_block_supp_list]


@app.callback(
    Output("Quoted_Price_Comparison", "figure"),
    [Input("supplier-dropdown", "value"),
     Input("product_block_dropdown", "value"),
     Input("TPNB_dropdown", "value")]
)
def Quoted_Price_Comparison(iSupplier, iBlock, iTPNB):
    pivot_Quoted_Price_Comparison = df[(df['supplier_name'] == iSupplier) & (df['product_block'].isin(iBlock)) & ~(df['cost_component'] == 'Delivered Cost') &(df['TPNB'].isin(iTPNB))].groupby('cost_component').mean()[['quoted_cost', 'best_bid']]
    pivot_Quoted_Price_Comparison.reset_index(inplace=True)
    cost_component = list(pivot_Quoted_Price_Comparison['cost_component'])
    quoted_cost = list(pivot_Quoted_Price_Comparison['quoted_cost'])
    best_bid = list(pivot_Quoted_Price_Comparison['best_bid'])

    fig = go.Figure()
    fig.add_trace(go.Bar(x=cost_component, y=quoted_cost, name='Quoted Cost', marker_color='rgb(55, 83, 109)'))
    fig.add_trace(go.Bar(x=cost_component, y=best_bid, name='Best Bid', marker_color='rgb(26, 118, 255)'))

    fig.update_layout(
        title='Price Breakdown Comparison',
        xaxis_tickfont_size=8,
        xaxis=dict(
            title='Cost Component',
            titlefont_size=12,
            tickfont_size=10,
        ),
        legend=dict(
            x=0.6,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        bargap=0.15,  # gap between bars of adjacent location coordinates.
        bargroupgap=0.1  # gap between bars of the same location coordinate.
    )
    return fig


@app.callback(
    Output("Total_Price_Comparison", "figure"),
    [Input("supplier-dropdown", "value"),
     Input("product_block_dropdown", "value"),
     Input("TPNB_dropdown", "value")]
)
def Total_Price_Comparison(iSupplier, iBlock, iTPNB):
    pivot_Total_Price_Comparison = df[(df['supplier_name'] == iSupplier) &(df['product_block'].isin(iBlock)) & ~(df['cost_component'] == 'Delivered Cost') &(df['TPNB'].isin(iTPNB))].groupby('cost_component').mean()[['total_cost', 'best_total_cost']]
    pivot_Total_Price_Comparison.reset_index(inplace=True)
    cost_component = list(pivot_Total_Price_Comparison['cost_component'])
    total_cost = list(pivot_Total_Price_Comparison['total_cost'])
    best_total_cost = list(pivot_Total_Price_Comparison['best_total_cost'])

    fig = go.Figure()
    fig.add_trace(go.Bar(x=cost_component, y=total_cost, name='Total Cost', marker_color='rgb(55, 83, 109)'))
    fig.add_trace(go.Bar(x=cost_component, y=best_total_cost, name='Best Total Cost', marker_color='rgb(26, 118, 255)'))

    fig.update_layout(
        title='Total Cost Breakdown Comparison',
        xaxis_tickfont_size=8,
        xaxis=dict(
            title='Cost Component',
            titlefont_size=12,
            tickfont_size=10,
        ),
        legend=dict(
            x=0.6,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        bargap=0.15,  # gap between bars of adjacent location coordinates.
        bargroupgap=0.1  # gap between bars of the same location coordinate.
    )
    return fig

@app.callback(
    Output("Del_Cost_PB", "data"),
    [Input("pb_dropdown", "value")]
)
def Del_Cost_PB(iPBlock):
    pivot_Del_Cost_PB = pd.pivot_table(df[(df['product_block'].isin(iPBlock)) & 
    (df['cost_component'] == 'Delivered Cost')], values='quoted_cost_wgt', index=['product_block'],
    columns=['supplier_name'], aggfunc=np.sum)
    pivot_Del_Cost_PB.replace(0, np.nan,inplace=True)
    pivot_Del_Cost_PB['Minimum'] = pivot_Del_Cost_PB[pivot_Del_Cost_PB.columns].min(axis=1)
    pivot_Del_Cost_PB.reset_index(inplace=True)
    pivot_Del_Cost_PB.rename(columns={'product_block':'Product Block'},inplace=True)

    return pivot_Del_Cost_PB.round(2).to_dict('records')

@app.callback(
    Output("Total_Cost_PB", "data"),
    [Input("pb_dropdown", "value")]
)
def Total_Cost_PB(iPBlock):
    pivot_Total_Cost_PB = pd.pivot_table(df[(df['product_block'].isin(iPBlock)) & 
    (df['cost_component'] == 'Delivered Cost')], values='total_cost', index=['product_block'],
    columns=['supplier_name'], aggfunc=np.sum)
    pivot_Total_Cost_PB.replace(0, np.nan,inplace=True)
    pivot_Total_Cost_PB['Minimum'] = pivot_Total_Cost_PB[pivot_Total_Cost_PB.columns].min(axis=1)
    pivot_Total_Cost_PB.reset_index(inplace=True)
    pivot_Total_Cost_PB.rename(columns={'product_block':'Product Block'},inplace=True)

    return pivot_Total_Cost_PB.round(2).to_dict('records')

@app.callback(
    Output("Savings_PB", "data"),
    [Input("pb_dropdown", "value")]
)
def Savings_PB(iPBlock):
    pivot_Savings_PB = pd.pivot_table(df[(df['product_block'].isin(iPBlock)) & 
    (df['cost_component'] == 'Delivered Cost') & (df['Is_TPNB'] == 1) &
    (df['Is_NPD'] == 0)], values='Savings', index=['product_block'],
    columns=['supplier_name'], aggfunc=np.sum)
    pivot_Savings_PB.replace(0, np.nan,inplace=True)
    pivot_Savings_PB['Minimum'] = pivot_Savings_PB[pivot_Savings_PB.columns].min(axis=1)
    pivot_Savings_PB.reset_index(inplace=True)
    pivot_Savings_PB.rename(columns={'product_block':'Product Block'},inplace=True)

    return pivot_Savings_PB.round(2).to_dict('records')

@app.callback(
    Output("supplier_feedback_grid", "data"),
    [Input("supplier-checklist", "value")]
)
def supplier_feedback_grid(iSupplier):
    cols=['product_block','TPNB','description','volume','Supplier_Quantile_Rank']
    df2=df[(df['supplier_name']==iSupplier)&(df['cost_component']=='Delivered Cost')].drop_duplicates()
    df2=df2[cols]
    df3=df2.groupby(['product_block','TPNB','description','volume'])['Supplier_Quantile_Rank'].sum().reset_index()
    df3['Position']=np.where(df3['Supplier_Quantile_Rank']==0,'NOT BID',
                            np.where((df3['Supplier_Quantile_Rank']>0) & (df3['Supplier_Quantile_Rank']<=2),'COMPETITIVE','NON COMPETITIVE'))
    df4=df3.sort_values(by=['product_block','TPNB'],ascending=False)
    df4.columns=['Product Block', 'TPNB', 'Description', 'Volume','Supplier Quantile Rank', 'Position']
    
    return df4.round(2).to_dict('records')




app.run_server()



NameError: name 'df' is not defined

In [23]:
cols=['supplier_name','product_block','TPNB','description','Is_NPD','Is_TPNB','cost_component','current_cost','quoted_cost','best_bid','best_supplier','volume','Impact']
df_sup=df[cols].drop_duplicates()
df_sup_2=df_sup[(df['supplier_name']=='Mikado UK')&(df_sup['Is_NPD']==0)&(df_sup['Is_TPNB']==1)]
pivot = df_sup_2.groupby(['TPNB','cost_component'],as_index=False).agg({
    'current_cost':'sum',
    'quoted_cost':'sum',
    'best_bid':'sum',
    'Impact':'sum'

}).reset_index(drop = True)
df_2=pivot.sort_values(by=['Impact'],ascending=False).head(14)
df3=df_2[['TPNB']].merge(df_sup[df_sup['supplier_name']=='Mikado UK'],how='inner',left_on=['TPNB'],right_on=['TPNB'])
cols=['product_block','TPNB','description','cost_component','current_cost','quoted_cost','best_bid','best_supplier','volume','Impact']
df4=df3[cols].drop_duplicates()
df5=df4.sort_values(by=['Impact'],ascending=False).head(14)
df5['Impact']=df5['Impact'].round(1)
new_cols=['Product Block','TPNB','Description','Cost Component','Current Cost','Quoted Cost','Best Bid','Best Supplier','Volume','Impact']
df5.columns=new_cols
df5

,Product Block,TPNB,Description,Cost Component,Current Cost,Quoted Cost,Best Bid,Best Supplier,Volume,Impact
0,Malt Vinegar,1,HS Distilled Malt Vinegar PM45 284ml,Delivered Cost,0.188333,0.20,0.20,Baxters 1,108672,0.0
1,Malt Vinegar,1,HS Distilled Malt Vinegar PM45 284ml,A:Raw Materials Cost,0.188333,0.08,0.08,Mikado UK,108672,0.0
70,Specialty Vinegar,83344526,TESCO CIDER VINEGAR 350ML,C2:Packaging Cost,0.490000,0.02,0.02,Mikado UK,678610,0.0
69,Specialty Vinegar,83344526,TESCO CIDER VINEGAR 350ML,C1:Processing Cost,0.490000,0.02,0.02,Mikado UK,678610,0.0
68,Specialty Vinegar,83344526,TESCO CIDER VINEGAR 350ML,B:Inbound Logistics,0.490000,0.02,0.02,Mikado UK,678610,0.0
67,Specialty Vinegar,83344526,TESCO CIDER VINEGAR 350ML,A:Raw Materials Cost,0.490000,0.23,0.23,Mikado UK,678610,0.0
66,Specialty Vinegar,83344526,TESCO CIDER VINEGAR 350ML,Delivered Cost,0.490000,0.33,0.33,Mikado UK,678610,0.0
47,Malt Vinegar,85244682,JACK'S MALT VINEGAR 568ML,D:Direct Distrb Cost,0.268333,0.03,0.03,Mikado UK,12364,0.0
46,Malt Vinegar,85244682,JACK'S MALT VINEGAR 568ML,C2:Packaging Cost,0.268333,0.02,0.02,Mikado UK,12364,0.0
45,Malt Vinegar,85244682,JACK'S MALT VINEGAR 568ML,C1:Processing Cost,0.268333,0.02,0.02,Mikado UK,12364,0.0


In [2]:
df.head(2)

,Phase,supplier_name,product_block,TPNB,description,Is_TPNB,Is_NPD,Savings,cost_component,quoted_cost,...,current_cost,current_cost_wgt,negotiation_index,negotiation_value,No_Bids,Impact,TPN_component,Supplier_Quantile,Supplier_Quantile_Rank,buyer
0,Phase1,Baxters 1,Balsamic Vinegar,51115622,TESCO BALSAMIC VINEGAROF MODENA 250ML,0,0.0,-636572.72,Delivered Cost,0.0,...,0.49,0.304512,0.408528,530730.423659,7,-597598.88,51115622-Delivered Cost,0.125,1.0,Cooking Ingredients
1,Phase1,Baxters 1,Balsamic Vinegar,51115622,TESCO BALSAMIC VINEGAROF MODENA 250ML,0,0.0,-636572.72,A:Raw Materials Cost,NaN,...,0.49,0.304512,0.408528,530730.423659,7,0.00,51115622-A:Raw Materials Cost,NaN,NaN,Cooking Ingredients


In [14]:
cols=['supplier_name','product_block','TPNB','volume','negotiation_index','current_cost','No_Bids','best_supplier','Supplier_Quantile_Rank']
df2=df[cols][(df['supplier_name']=='LDH Ltd')&(df['cost_component']=='Delivered Cost')&(df['Is_TPNB']==1)].drop_duplicates()

product_block_count=df2.product_block.nunique()

df3=df2.groupby(['supplier_name'])['product_block','TPNB'].nunique().reset_index()
df3

df['TPNB'].nunique()

df3['Total_Product_Bid%']=round((df3['TPNB']/df['TPNB'].nunique())*100,0)
df3

upper_quartile=df2[(df2['Supplier_Quantile_Rank'].isin([1,2]))&(df2['supplier_name']=='LDH Ltd')]
upper_quartile=upper_quartile.groupby(['product_block'])['Supplier_Quantile_Rank'].count().reset_index()
upper_quartile

lower_Quartile=df2[(df2['Supplier_Quantile_Rank'].isin([3,4]))&(df2['supplier_name']=='LDH Ltd')]
lower_Quartile=lower_Quartile.groupby(['product_block'])['Supplier_Quantile_Rank'].count().reset_index()
lower_Quartile

quartile=upper_quartile.merge(lower_Quartile,on=['product_block'],how='outer')

quartile['upper_bid%']=((quartile['Supplier_Quantile_Rank_x'])/(quartile['Supplier_Quantile_Rank_x']+quartile['Supplier_Quantile_Rank_y']))*100
quartile

tpnb_bid=df2.groupby(['product_block'])['TPNB'].nunique().reset_index()
tpnb_bid

total_tpnb=df.groupby(['product_block'])['TPNB'].nunique().reset_index()
total_tpnb

bid_per=tpnb_bid.merge(total_tpnb,how='inner',on=['product_block'])

bid_per['Bid_%']=(bid_per['TPNB_x']/bid_per['TPNB_y'])*100
bid_per

product_block_position=quartile.merge(bid_per[['product_block','Bid_%']],on=['product_block'],how='outer')
product_block_position

product_block_position['compititive']=np.where(((product_block_position['Bid_%']>=80) & (product_block_position['upper_bid%']>=25)),'Competitive',
                                         np.where(((product_block_position['Bid_%']>=60) & (product_block_position['upper_bid%']>=40)),'Competitive_2','Non Competitive'))



product_block_position['text']=np.where(product_block_position['compititive']=='Competitive','You bid on more than 80% of the block and more than 25% of your bids were competitive',
                                 np.where(product_block_position['compititive']=='Competitive_2','You bid on less than 60% of the block and less than 40% of them were competitive','You bid on more than 80% of the block but less than 25% of your bids were competitive'))

product_block_position

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,product_block,Supplier_Quantile_Rank_x,Supplier_Quantile_Rank_y,upper_bid%,Bid_%,compititive,text
0,Balsamic Vinegar,3.0,2,60.0,100.0,Competitive,You bid on more than 80% of the block and more...
1,Specialty Vinegar,NaN,6,NaN,100.0,Non Competitive,You bid on more than 80% of the block but less...


In [120]:

upper_quartile_tpnb=sum(upper_quartile['Supplier_Quantile_Rank'])

In [121]:

lower_quartile_tpnb=sum(lower_Quartile['Supplier_Quantile_Rank'])

In [122]:
top_2_quartile_per=(upper_quartile_tpnb/(upper_quartile_tpnb+lower_quartile_tpnb))*100

In [123]:
df3=df2.groupby(['supplier_name'])['product_block','TPNB'].nunique().reset_index()
df3

df['TPNB'].nunique()

df3['Total_Product_Bid%']=round((df3['TPNB']/df['TPNB'].nunique())*100,0)
df3

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,supplier_name,product_block,TPNB,Total_Product_Bid%
0,Baxters 1,2,12,67.0


In [124]:
df3['Overall_position']=np.where(((df3['Total_Product_Bid%']>60)&(top_2_quartile_per>25)),'Competitive',
                                 np.where(((df3['Total_Product_Bid%']>40)&(top_2_quartile_per>40)),'Competitive_2','Non Competitive'))

In [125]:
df3

,supplier_name,product_block,TPNB,Total_Product_Bid%,Overall_position
0,Baxters 1,2,12,67.0,Non Competitive
